This notebook is a first attempt to translate the FORTRAN90 code from MTV2016 into python
and to replicate the main results of this paper.

In [1]:
import numpy as np
from scipy.stats import lognorm
from numba import jit

import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

In [2]:
#Paramters for the piece-rate version

delta = 0.000226
lamb_e = 0.7048178
k = 22.9931
sigma_z = 0.0
rhorho1 = 4.296953
rho2 = 0.065518
zlearningprob = 0.26398515
b = 2.3105359
theta9 = 0.0
theta10 = 0.0
zshockprob = 0.0093854   


#    IF (npar>=10) THEN
#    rho3=theta(9)  
#    rho4=theta(10)  
#    ELSE IF (npar==8) THEN
#    rho3=0.0_8
#    rho4=0.0_8
#    ELSE 

In [3]:
# sigma_z = 0.2
# mu_temp = np.exp(-0.5 * sigma_z**2) # this guarantess that the mean of the distribution is 1

# x = np.linspace(0, 4, 100)
# plt.plot(x, lognorm.cdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.plot(x, lognorm.pdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.show()

In [4]:
# Match shock z.
# First, populating the match productivity shock vector, if zpts_first is >1

zpts_first = 51 # basic grid. 102 in MTV2016
zpts = zpts_first

# zvector are the actual values of the match quality, dim(zpts)
# ztransmatrix is the transition matrix, dim(zpts, zpts)

# Obtain a discrete LOGNORMAL probability distr for z

zprobcdf = np.zeros((zpts))
zprobpdf = np.zeros((zpts))
zvector = np.zeros((zpts))

sigma_z = 0.2 # CHANGE OF PARAMTER: sigma_z must be positive
mu_temp = np.exp(-0.5 * sigma_z**2) # mean of the lognormal distr, s.t. mean of the distr is 1

shape = sigma_z

resolution = 1000 # Resolution should be much higher than no. of bins (resolution=10000)
xstep = 1 / resolution

xvalue = xstep # move along the interval of z's, starting close to 1
tempreal = 0.5 * xstep * lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # this value is used to weight the z-values
# properly within the bin
bincounter = 1 # start with the first bin
binsize = 1 / (zpts - 1) # binsize

# zprobcdf contains the probabilities of the idiosyncratic shock, dimension zpts

# First fill all but the last coordinate
while bincounter < zpts - 1:
    if lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= ((bincounter + 0) * binsize): # if the bin is full..
        zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # add to the cdf
        if bincounter == 1:
            zprobpdf[bincounter] = zprobcdf[bincounter]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        else:
            zprobpdf[bincounter] = zprobcdf[bincounter] - zprobcdf[bincounter - 1]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        
        tempreal = 0 # reset the average counter 
        bincounter += 1
        
#         while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= (bincounter * binsize):
#             zvector[bincounter] = xstep
#             zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp)
#             zprobpdf[bincounter] = 0
#             bincounter += 1
            
    else:
        while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) < ((bincounter + 0) * binsize):
            xvalue = xvalue + xstep
            tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
# # last gridpoint

zprobcdf[zpts-1] = 1
zprobpdf[zpts-1] = 1 - zprobcdf[zpts - 2]

tempreal = 0
xstep = xvalue / (zpts * 5)

counter1 = 1
while counter1 < 11:
    xvalue = xvalue + xstep
    tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
    counter1 += 1
    
zvector[zpts-1] = tempreal / (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) - zprobcdf[zpts - 2])

In [5]:
# Z-PRODUCTIVITY TRANSITION MATRICES

# INITIAL LEARNING, same for all options

ztransmatrix = np.zeros((zpts, zpts))
zlearningprob = 0.26398515 # from MTV2016

ztransmatrix[0, 0] = 0
for ind in range(1, zpts):
    ztransmatrix[0, ind] = zlearningprob * zprobpdf[ind]
    
tempreal = np.sum(ztransmatrix[0, :])
ztransmatrix[0, 0] = 1 - tempreal

# Permanent z's once learned
for ind in range(1, zpts):
    ztransmatrix[ind, ind] = 1

In [6]:
# Worker productivity - ALL WORKER IDENTICAL WHEN BORN. Otherwise, set ypts = 20 (MTV2016 sub-option)

tmax = 576 # 65-18=48 years, i.e. 48*12=576 months
ypts = 1

y = np.zeros((ypts))
yprobcdf = np.zeros((ypts))
yprobpdf = np.zeros((ypts))
prod = np.zeros((ypts, tmax))

y[:] = 1
yprobcdf[0] = 1
yprobpdf[0] = 1

@jit
def pfunction(exper, rhorho1, rho2):
    output = 1 - rhorho1 + (rhorho1 * exper + 1)**rho2
    return output

# ALLOCATE WORKER PRODUCTIVITY MATRIX, which incorporates both human capital and  worker idiosyncratic productivity
for ycnt in range(0, ypts):
    for exper in range(0, tmax):
        prod[ycnt, exper] = y[ycnt] * pfunction(exper, rhorho1, rho2)  

In [10]:
# Discrete MArkov chain approximation of continuous AR(1) process of aggregate productivity 

from quantecon import tauchen

rho = 0.7 # autocorrelation of AR(1) (Wee)
sigma_u = 0.0165 # standard deviation of AR(1), (Shimer 2005)

prod = tauchen(rho, sigma_u, m=3, n=3)

# prod.P # shows transition matrix
# prod.state_values # associated state space
# prod.stationary_distributions # shows the stationary distribution

In [ ]:
beta # discount factor (0.996737)
b # value of leisure (2.3105359)

zprobpdf #prob. distribution of match quality
sigma # scale of zprobpdf
phi # shape of zprobpdf
eta # prob. that match quality changes
alpha # prob. that match quality is observed

k # vacancy cost (22.9931)
lambda_u = 1 # prob. unemployed worker gets to search
lambda_e # prob. employed worker gets to search (0.7048178)
delta # separation prob. (0.000226)

mu # vector of age t entries into the labour market
nu # vector of age t exits from the labour market

In [ ]:
# Initialize



In [31]:
ppts = 1 # no aggregate shock in MTV2016
ypts = 1 # no ex-ante heterogeneity in MTV016
tmx = 576
expts = tmax

VU = np.zeros((ppts, ypts, expts, tmax))        # Value of unemployment
VE = np.zeros((ppts, ypts, expts, zpts, tmax))  # Joint value of employment, conditional on z, t

# STILL NEEDS TO BE TRANSLATED INTO PYTHON
poliDW[ppts, ypts, 0:expts, zpts, 0:tmax] = 0
Dmax[ppts, ypts, 0:expts, zpts,2] = 0 # Expected  surplus value of search. WATCH OUT: second argument zpts refers to current promised value
DmaxU[ppts, ypts, 0:expts, 2] = 0 # Expected surplus value of search for unemployed
S[ppts, ypts, 0:expts, zpts, 2] = 0 # Expected productivity of the match
phiu[ppts, ypts, 0:expts, 0:tmax] = 0 # piece rate offered to previously unemployed
phie[ppts, ypts, 0:expts, zpts, 0:tmax] = 0 # piece rate offered to previously employed at z
duration[ppts, ypts,0:expts, zpts, 2] = 0 # duration, match with current z, not necess. initial z
JF_E[ppts, ypts, 0:expts, zpts, 0:tmax] = 0 # Prob of Succesful application *from* a match z,t
JF_U[ppts, ypts, 0:expts, 0:tmax] = 0 # Prob of Succesful application *from* a match unemployment at t

In [ ]:
# LAST PERIOD, t=T

VU[:, :, :, 2] = b